In [2]:
import typing
from fsr_trainable import FSR_Trainable
import fsr_data
import fsr_predictor
import numpy as np
import warnings
import ray, logging
import ray.tune
np.seterr(divide='ignore', invalid='ignore')
display(ray.init(log_to_driver=False, logging_level=logging.ERROR, ignore_reinit_error=True))
warnings.filterwarnings("ignore", category=DeprecationWarning)

Python version:,3.10.12
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [3]:
class Tasks:
    TASKS = [
        'FSR_Trainable_2023-08-10_14-56-54',
        'FSR_Trainable_2023-08-10_16-28-50',
        'FSR_Trainable_2023-08-10_18-18-32',
        'FSR_Trainable_2023-08-11_03-36-28',
        'FSR_Trainable_2023-07-19_00-13-04',
        'FSR_Trainable_2023-07-19_01-25-28',
        'FSR_Trainable_2023-07-19_02-32-25',
        'FSR_Trainable_2023-07-19_03-37-48',
        'FSR_Trainable_2023-07-19_04-38-35',
        'FSR_Trainable_2023-08-10_22-35-32',
        'FSR_Trainable_2023-08-10_23-25-39',
        'FSR_Trainable_2023-08-11_00-39-35',
        'FSR_Trainable_2023-07-19_09-38-35',
        'FSR_Trainable_2023-07-19_10-04-48',
        'FSR_Trainable_2023-07-19_11-36-24',
        'FSR_Trainable_2023-07-19_12-32-34',
        'FSR_Trainable_2023-07-19_13-03-12',
        'FSR_Trainable_2023-08-07_15-54-13',
    ]

    def __call__(self, model:typing.Literal['ANN', 'CNN-LSTM', 'LSTM'], task:int):
        assert 1 <= task <= 6
        assert model in ['ANN', 'CNN-LSTM', 'LSTM']
        model_index = {'ANN':0, 'CNN-LSTM':1, 'LSTM':2}[model]
        return Tasks.TASKS[model_index + (task - 1) * 3]

    def __iter__(self):
        return iter(Tasks.TASKS)
    
    def __getitem__(self, index):
        return Tasks.TASKS[index]
TASKS = Tasks()

In [4]:
data = fsr_data.get_data()

In [5]:
def get_GRF(data):
    data = np.array(data)
    return data.sum(axis=1)
GRT = get_GRF(data['force'])

In [54]:
def get_CoP(force, x_coord, y_coord):
    force = np.array(force)
    x_coord = np.array(x_coord)
    y_coord = np.array(y_coord)

    # strict check
    # assert ((force == 0) == np.isnan(x_coord)).all()
    # assert ((force == 0) == np.isnan(y_coord)).all()
    # valid_check = ((force == 0) == (np.isnan(x_coord))) & ((force == 0) == (np.isnan(x_coord)))
    # x, y = np.where(valid_check == False)
    # display(data.loc[x, [('force', 'D'), ('x_coord', 'D')]])

    # weak check
    assert not ((force != 0) & (np.isnan(x_coord))).any()
    assert not ((force != 0) & (np.isnan(y_coord))).any()

    x_nan = np.isnan(x_coord)
    y_nan = np.isnan(y_coord)
    x_coord[x_nan] = 0
    y_coord[y_nan] = 0

    x_force = force
    y_force = force.copy()
    x_force[x_nan] = 0
    y_force[y_nan] = 0
    
    with np.errstate(divide='ignore'):
        x = (x_force * x_coord).sum(axis=1) / x_force.sum(axis=1)
        y = (y_force * y_coord).sum(axis=1) / y_force.sum(axis=1)
    
    assert (np.isnan(x) == np.isnan(y)).all()
    return np.stack([x, y], axis=0)

COPX, COPY = get_CoP(data['force'], data['x_coord'], data['y_coord'])

In [82]:
FSR_CENTER = np.array([
    (15*0.5, 14*0.5),
    (15*0.5, 24*0.5),
    (7*0.5, 24*0.5),
    (11*0.5, 51*0.5),
    (4.4, 7),
    (7.5, 18.5)
])
FSR_CENTER_COP = FSR_CENTER.mean(axis=0)
grfs = []
cops = []

for task in [1, 4]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        path = '/home/seokj/ray_results/' + TASKS(model, task)
        result_grid = ray.tune.Tuner.restore(path, FSR_Trainable).get_results()
        for result in result_grid:
            if result.best_checkpoints:
                checkpoint, metric = result.best_checkpoints[-1]
                trainer = FSR_Trainable(config=metric['config'])
                trainer.restore(checkpoint)
                output = trainer.eval()
                preds = []
                ys = []
                for pred, y in output:
                    preds.append(pred)
                    ys.append(y)
                preds = np.concatenate(preds)
                ys = np.concatenate(ys)

                # data definition
                pred_force = preds[:, :6]
                pred_x_coord = preds[:, 6:12]
                pred_y_coord = preds[:, 12:]
                y_force = ys[:, :6]
                y_x_coord = ys[:, 6:12]
                y_y_coord = ys[:, 12:]

                # get grf and cop
                pred_grf = get_GRF(pred_force)
                y_grf = get_GRF(y_force)
                pred_x_cop, pred_y_cop = get_CoP(pred_force, pred_x_coord, pred_y_coord)
                y_x_cop, y_y_cop = get_CoP(y_force, y_x_coord, y_y_coord)

                # fill nan
                pred_x_cop[np.isnan(pred_x_cop) & ~np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
                y_x_cop[~np.isnan(pred_x_cop) & np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
                pred_y_cop[np.isnan(pred_y_cop) & ~np.isnan(y_y_cop)] = FSR_CENTER_COP[1]
                y_y_cop[~np.isnan(pred_y_cop) & np.isnan(y_y_cop)] = FSR_CENTER_COP[1]

                # get rmse of grf and cop
                grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
                cop = ((pred_x_cop - y_x_cop) ** 2 + (pred_y_cop - y_y_cop) ** 2).mean(axis=0) ** 0.5
                grfs.append((grf, checkpoint.path, task, model))
                cops.append((cop, checkpoint.path, task, model))
                print(grf, cop)


search task: 1, model: ANN
not found optimizer state dict
624.673664804272 6.096413993514734
not found optimizer state dict
612.6985392507477 3.622275018859813
not found optimizer state dict
625.9532730164449 6.370250952551738
not found optimizer state dict
976.418231343516 4.337979002786122
not found optimizer state dict
1176.079929256511 11.142597810291551
not found optimizer state dict
564.9169463292812 8.083745362572603
not found optimizer state dict
640.513221955644 5.932640255227787
not found optimizer state dict
564.1601335170716 3.5878248380009117
not found optimizer state dict
564.2046946809287 3.566856665021884
not found optimizer state dict
798.652302319351 13.981797510373756
not found optimizer state dict
3301.592343097494 123.31195355935692
not found optimizer state dict
1224.2999530343861 8.780537097813221
not found optimizer state dict
635.7529492656719 9.34612414967262
not found optimizer state dict
1078.4801226726433 856.6989334065964
not found optimizer state dict
127

In [93]:
from fsr_predictor import get_best_checkpoint

for task in [2, 5]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        try:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse_force', 'min')
        except:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse', 'min')
        trainer = FSR_Trainable(config=metric['config'])
        trainer.restore(checkpoint)
        output = trainer.eval()
        preds = []
        ys = []
        for pred, y in output:
            preds.append(pred)
            ys.append(y)
        preds = np.concatenate(preds)
        ys = np.concatenate(ys)

        # data definition
        pred_force = preds
        y_force = ys

        # get grf and cop
        pred_grf = get_GRF(pred_force)
        y_grf = get_GRF(y_force)

        # get rmse of grf and cop
        grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
        grfs.append((grf, checkpoint.path, task, model))
        print(grf)


search task: 2, model: ANN
not found optimizer state dict
565.010287516962
search task: 2, model: CNN-LSTM
not found optimizer state dict
547.3218602888066
search task: 2, model: LSTM
not found optimizer state dict
443.8692797434848
search task: 5, model: ANN
not found optimizer state dict
575.2720008830605
search task: 5, model: CNN-LSTM
not found optimizer state dict
416.0659315240795
search task: 5, model: LSTM
not found optimizer state dict
446.4606470619779


In [107]:
from fsr_predictor import get_best_checkpoint, get_best_prediction

experiment = min(((e[0], e[1]) for e in grfs if e[2] == 2), key=lambda e: e[1])[1].split('/')[-3]
try:
    output = get_best_prediction(experiment, 'rmse_force', 'min')
except:
    output = get_best_prediction(experiment, 'rmse', 'min')
BEST_GRF_TASK3_PRED = np.concatenate([e[0] for e in output])
BEST_GRF_TASK3_Y = np.concatenate([e[1] for e in output])

experiment = min(((e[0], e[1]) for e in grfs if e[2] == 5), key=lambda e: e[1])[1].split('/')[-3]
try:
    output = get_best_prediction(experiment, 'rmse_force', 'min')
except:
    output = get_best_prediction(experiment, 'rmse', 'min')
BEST_GRF_TASK6_PRED = np.concatenate([e[0] for e in output])
BEST_GRF_TASK6_Y = np.concatenate([e[1] for e in output])

not found optimizer state dict
not found optimizer state dict


In [109]:
for task in [3, 6]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        try:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse_coord', 'min')
        except:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse', 'min')
        trainer = FSR_Trainable(config=metric['config'])
        trainer.restore(checkpoint)
        output = trainer.eval()
        preds = []
        ys = []
        for pred, y in output:
            preds.append(pred)
            ys.append(y)
        preds = np.concatenate(preds)
        ys = np.concatenate(ys)

        # data definition
        pred_force = BEST_GRF_TASK3_PRED if task == 3 else BEST_GRF_TASK6_PRED
        pred_x_coord = preds[:, :6]
        pred_y_coord = preds[:, 6:]
        y_force = BEST_GRF_TASK3_Y if task == 3 else BEST_GRF_TASK6_Y
        y_x_coord = ys[:, :6]
        y_y_coord = ys[:, 6:]

        # get grf and cop
        pred_grf = get_GRF(pred_force)
        y_grf = get_GRF(y_force)
        pred_x_cop, pred_y_cop = get_CoP(pred_force, pred_x_coord, pred_y_coord)
        y_x_cop, y_y_cop = get_CoP(y_force, y_x_coord, y_y_coord)

        # fill nan
        pred_x_cop[np.isnan(pred_x_cop) & ~np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
        y_x_cop[~np.isnan(pred_x_cop) & np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
        pred_y_cop[np.isnan(pred_y_cop) & ~np.isnan(y_y_cop)] = FSR_CENTER_COP[1]
        y_y_cop[~np.isnan(pred_y_cop) & np.isnan(y_y_cop)] = FSR_CENTER_COP[1]

        # get rmse of grf and cop
        grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
        cop = ((pred_x_cop - y_x_cop) ** 2 + (pred_y_cop - y_y_cop) ** 2).mean(axis=0) ** 0.5
        grfs.append((grf, checkpoint.path, task, model))
        cops.append((cop, checkpoint.path, task, model))
        print(grf, cop)


search task: 3, model: ANN
not found optimizer state dict
547.3218602888066 424.98825351414126
search task: 3, model: CNN-LSTM
not found optimizer state dict
547.3218602888066 345.42667230180706
search task: 3, model: LSTM
not found optimizer state dict
547.3218602888066 359.62967063355603
search task: 6, model: ANN
not found optimizer state dict
575.2720008830605 4.695813347481431
search task: 6, model: CNN-LSTM
not found optimizer state dict
575.2720008830605 4.7150395438840835
search task: 6, model: LSTM
not found optimizer state dict
575.2720008830605 4.7665952601564054


In [112]:
best_grf_model = {}
best_cop_model = {}
for task in range(1, 7):
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        best_grf_model[(task, model)] = min(((e[0], e[1]) for e in grfs if e[2] == task and e[3] == model), key=lambda e: e[1])
        if task not in [2, 5]:
            best_cop_model[(task, model)] = min(((e[0], e[1]) for e in cops if e[2] == task and e[3] == model), key=lambda e: e[1])
display(best_grf_model)
display(best_cop_model)

{(1, 'ANN'): (624.673664804272,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_14-56-54/FSR_Trainable_00208eb6_89_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_15-17-42/checkpoint_000065'),
 (1, 'CNN-LSTM'): (667.4809922387303,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_16-28-50/FSR_Trainable_072855d3_73_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_16-59-19/checkpoint_000100'),
 (1, 'LSTM'): (612.4154533484601,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_18-18-32/FSR_Trainable_026edc0a_42_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_18-40-04/checkpoint_000085'),
 (2, 'ANN'): (565.010287516962,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-11_03-36-28/FSR_Trainable_12e4a933_5_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,index

{(1, 'ANN'): (6.096413993514734,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_14-56-54/FSR_Trainable_00208eb6_89_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_15-17-42/checkpoint_000065'),
 (1, 'CNN-LSTM'): (47.53153250244778,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_16-28-50/FSR_Trainable_072855d3_73_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_16-59-19/checkpoint_000100'),
 (1, 'LSTM'): (60.01963383187621,
  '/home/seokj/ray_results/FSR_Trainable_2023-08-10_18-18-32/FSR_Trainable_026edc0a_42_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-08-10_18-40-04/checkpoint_000085'),
 (3, 'ANN'): (424.98825351414126,
  '/home/seokj/ray_results/FSR_Trainable_2023-07-19_02-32-25/FSR_Trainable_01ccd703_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,im